In [1]:
import requests
from bs4 import BeautifulSoup
import datetime

# 获取页面
def get_page(link):
    headers = {'User-Agent' : 'Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US; rv:1.9.1.6) Gecko/20091201 Firefox/3.5.6'} 
    r = requests.get(link, headers = headers)
    html = r.content  #使用r.content解封装
    html = html.decode('utf-8')  #由UTF-8解码为unicode
    soup = BeautifulSoup(html, 'lxml')
    return soup

# 解析网页
def get_data(post_list):
    data_list =[]
    for post in post_list:
        title = post.find('div',class_='titlelink box').text.strip()
        post_link = post.find('div',class_='titlelink box').a['href']
        post_link = "https://bbs.hupu.com" + post_link 

        author = post.find('div',class_='author box').a.text.strip()
        author_page = post.find('div',class_='author box').a['href']
        start_date = post.find('div',class_='author box').contents[5].text.strip()

        reply_view = post.find('span',class_='ansour box').text.strip()
        reply = reply_view.split('/')[0].strip()
        view = reply_view.split('/')[1].strip()

        reply_time = post.find('div',class_='endreply box').a.text.strip()
        last_reply = post.find('div',class_='endreply box').span.text.strip()
        if ':' in reply_time: #时间是11:27
            date_time = str(datetime.date.today()) + ' ' + reply_time
            date_time = datetime.datetime.strptime(date_time, '%Y-%m-%d %H:%M')
        elif reply_time.find("-") == 4: #时间是2017-02-27
            date_time = datetime.datetime.strptime(reply_time, '%Y-%m-%d').date()
        else: #时间是11-27
            date_time = datetime.datetime.strptime('2018-' + reply_time, '%Y-%m-%d').date()
        data_list.append([title, post_link, author, author_page, start_date, reply, view, last_reply, date_time])
    return data_list

link = "https://bbs.hupu.com/bxj"
soup = get_page(link)
post_all= soup.find('ul', class_="for-list")
post_list = post_all.find_all('li')
data_list = get_data(post_list)
for each in data_list:
    print (each)

['[置顶]\n【虎扑✖任天堂】来聊你和NS的故事，赢海量Q币红包雨！\n\n\xa0\n\n[\xa0\n2\n3...20\n\n\xa0]', 'https://bbs.hupu.com/39825238.html', '虎扑的步行街', 'https://my.hupu.com/260395651841167', '2020-12-17', '393', '3341476', '自然醒x', datetime.datetime(2020, 12, 20, 16, 45)]
['给需要的jr\n\n\xa0\n\n[\xa0\n2\n3...9\n\n\xa0]', 'https://bbs.hupu.com/39776759.html', '丶不甘', 'https://my.hupu.com/265768964123814', '2020-12-14', '163', '77698', '恩ono呐', datetime.datetime(2020, 12, 20, 18, 48)]
['各位大佬救救孩子吧', 'https://bbs.hupu.com/39884111.html', 'KING_ONE', 'https://my.hupu.com/7954895983329', '2020-12-20', '1', '2', 'KING_ONE', datetime.datetime(2020, 12, 20, 18, 48)]
['国企研究所单休，领到竟然让我做出这种事!\n\n\xa0\n\n[\xa0\n2\n3...8\n\n\xa0]', 'https://bbs.hupu.com/39881820.html', '质朴星人赖大强', 'https://my.hupu.com/115593883644691', '2020-12-20', '151', '84157', '卡梅罗杜兰特2', datetime.datetime(2020, 12, 20, 18, 48)]
['我就觉得自己比丁真帅，咋的？\n\n\xa0\n\n[\xa0\n2\n3...7\n\n\xa0]', 'https://bbs.hupu.com/39883164.html', '科比我爱1', 'https://my.hupu.com/26925626055155

In [6]:
from pymongo import MongoClient
class MongoAPI(object):
    def __init__(self, db_ip, db_port, db_name, table_name):
        self.db_ip = db_ip
        self.db_port = db_port
        self.db_name = db_name
        self.table_name = table_name
        self.conn = MongoClient(host=self.db_ip, port=self.db_port)
        self.db = self.conn[self.db_name]
        self.table = self.db[self.table_name]
    def get_one(self, query):
        return self.table.find_one(query, projection={"_id": False})
    def get_all(self, query):
        return self.table.find(query)
    def add(self, kv_dict):
        return self.table.insert_one(kv_dict)
    def delete(self, query):
        return self.table.delete_many(query)
    def check_exist(self, query):
        ret = self.table.find_one(query)
        return ret != None
    # 如果没有会新建
    def update(self, query, kv_dict):
            self.table.update_one(query,{
              '$set': kv_dict
            }, upsert=True)

In [13]:
hupu_post = MongoAPI("localhost",  27017,  "hupu", "post")
for each in data_list:
    hupu_post.add({"title": each[0], 
                "post_link": each[1],
               "author": each[2],
               "author_page": each[3],
               "start_date": str(each[4]),
               "reply": each[5],
               "view": each[6],
               "last_reply": each[7],
               "last_reply_time": str(each[8])})

{'title': '[置顶]\n【虎扑✖任天堂】来聊你和NS的故事，赢海量Q币红包雨！\n\n\xa0\n\n[\xa0\n2\n3...20\n\n\xa0]', 'post_link': 'https://bbs.hupu.com/39825238.html', 'author': '虎扑的步行街', 'author_page': 'https://my.hupu.com/260395651841167', 'start_date': '2020-12-17', 'reply': '393', 'view': '3341476', 'last_reply': '自然醒x', 'last_reply_time': '2020-12-20 16:45:00'}
{'title': '给需要的jr\n\n\xa0\n\n[\xa0\n2\n3...9\n\n\xa0]', 'post_link': 'https://bbs.hupu.com/39776759.html', 'author': '丶不甘', 'author_page': 'https://my.hupu.com/265768964123814', 'start_date': '2020-12-14', 'reply': '163', 'view': '77698', 'last_reply': '恩ono呐', 'last_reply_time': '2020-12-20 18:48:00'}
{'title': '各位大佬救救孩子吧', 'post_link': 'https://bbs.hupu.com/39884111.html', 'author': 'KING_ONE', 'author_page': 'https://my.hupu.com/7954895983329', 'start_date': '2020-12-20', 'reply': '1', 'view': '2', 'last_reply': 'KING_ONE', 'last_reply_time': '2020-12-20 18:48:00'}
{'title': '国企研究所单休，领到竟然让我做出这种事!\n\n\xa0\n\n[\xa0\n2\n3...8\n\n\xa0]', 'post_link': 'https

In [8]:
len(data_list)

121

In [36]:
import requests
from bs4 import BeautifulSoup
import datetime
from pymongo import MongoClient
import time

hupu_post = MongoAPI("localhost",  27017,  "hupu", "post")
for i in range(1,10):
    link = "https://bbs.hupu.com/bxj-" + str(i)
    soup = get_page(link)
    
    post_all= soup.find('div', class_="bbs-sl-web-post")
    post_list = post_all.find_all('li')
    data_list = get_data(post_list)
    for each in data_list:
        hupu_post.update({"post_link": each[1]},{"title": each[0], 
                    "post_link": each[1],
                   "author": each[2],
                   "author_page": each[3],
                   "start_date": str(each[4]),
                   "reply": each[5],
                   "view": each[6],
                   "last_reply": each[7],
                   "last_reply_time": str(each[8])})
    time.sleep(3)
    print ('第', i ,'页获取完成，休息3秒')

AttributeError: 'NoneType' object has no attribute 'text'

In [37]:
post_list

[<li class="bbs-sl-web-post-layout bbs-sl-web-post-body"><div class="post-title"><a class="p-title" href="/39825238.html" target="_blank">【虎扑✖任天堂】来聊你和NS的故事，赢海量Q币红包雨！</a><svg class="light-icon" height="15" p-id="3577" version="1.1" viewbox="0 0 1024 1024" width="15" xmlns="http://www.w3.org/2000/svg"><path d="M483.555556 0m28.444444 0l0 0q28.444444 0 28.444444 28.444444l0 56.888889q0 28.444444-28.444444 28.444445l0 0q-28.444444 0-28.444444-28.444445l0-56.888889q0-28.444444 28.444444-28.444444Z" fill="#191C22" p-id="3578"></path><path d="M540.444444 396.458667c0 14.563556-10.922667 26.396444-24.462222 26.624h-0.455111c-64.682667 0-117.873778 52.622222-124.131555 120.035555-0.455111 14.279111-11.434667 25.770667-24.917334 25.770667-13.539556 0-24.519111-11.491556-24.974222-25.770667H341.333333C347.704889 446.236444 423.253333 369.777778 515.527111 369.777778c13.767111 0 24.917333 11.946667 24.917333 26.680889z" fill="#FFFFFF" p-id="3579"></path><path d="M328.248889 828.131556l13.084444 8.

In [34]:
post_all[6]

<ul><li class="bbs-sl-web-post-layout bbs-sl-web-post-body"><div class="post-title"><a class="p-title" href="/39825238.html" target="_blank">【虎扑✖任天堂】来聊你和NS的故事，赢海量Q币红包雨！</a><svg class="light-icon" height="15" p-id="3577" version="1.1" viewbox="0 0 1024 1024" width="15" xmlns="http://www.w3.org/2000/svg"><path d="M483.555556 0m28.444444 0l0 0q28.444444 0 28.444444 28.444444l0 56.888889q0 28.444444-28.444444 28.444445l0 0q-28.444444 0-28.444444-28.444445l0-56.888889q0-28.444444 28.444444-28.444444Z" fill="#191C22" p-id="3578"></path><path d="M540.444444 396.458667c0 14.563556-10.922667 26.396444-24.462222 26.624h-0.455111c-64.682667 0-117.873778 52.622222-124.131555 120.035555-0.455111 14.279111-11.434667 25.770667-24.917334 25.770667-13.539556 0-24.519111-11.491556-24.974222-25.770667H341.333333C347.704889 446.236444 423.253333 369.777778 515.527111 369.777778c13.767111 0 24.917333 11.946667 24.917333 26.680889z" fill="#FFFFFF" p-id="3579"></path><path d="M328.248889 828.131556l13.084444